<a href="https://colab.research.google.com/github/FidelAHM/data_management_project/blob/main/Data_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#upload the csv
from google.colab import files

uploaded = files.upload()


Saving number_of_graduates_in_milan.csv to number_of_graduates_in_milan (1).csv


In [32]:
#getting the file name
filename = list(uploaded.keys())[0]
filename


'number_of_graduates_in_milan (1).csv'

In [33]:
# Load the CSV (Correct Separator)
import pandas as pd

df = pd.read_csv(filename, sep=",")
df.head()


,_id,ANNO_LAUREA,COD_Ateneo,NomeOperativo,NUMERO,NOME_CORSO,COMUNE,ID_PROVINCIA_ISTAT,ID_SESSO,laureati
0,1,2024,101,Torino,L/DS,SCIENZE STRATEGICHE E DELLA SICUREZZA,TORINO,1,F,24
1,2,2024,101,Torino,L/DS,Scienze Strategiche e della Sicurezza,TORINO,1,M,13
2,3,2024,101,Torino,L/SNT1,Infermieristica (Abilitante alla Professione S...,AOSTA,7,F,16
3,4,2024,101,Torino,L/SNT1,Infermieristica (abilitante alla professione s...,ASTI,5,F,45
4,5,2024,101,Torino,L/SNT1,Infermieristica (Abilitante alla Professione S...,ASTI,5,M,6


In [34]:
# Rename Columns to English for better understanding
df = df.rename(columns={
    "_id": "record_id",
    "ANNO_LAUREA": "graduation_year",
    "COD_Ateneo": "university_code",
    "NomeOperativo": "university_name",
    "NUMERO": "degree_class_code",
    "NOME_CORSO": "course_name",
    "COMUNE": "municipality",
    "ID_PROVINCIA_ISTAT": "province_istat_code",
    "ID_SESSO": "gender_code",
    "laureati": "graduates"
})

df.columns


Index(['record_id', 'graduation_year', 'university_code', 'university_name',
       'degree_class_code', 'course_name', 'municipality',
       'province_istat_code', 'gender_code', 'graduates'],
      dtype='object')

In [35]:
df.columns

Index(['record_id', 'graduation_year', 'university_code', 'university_name',
       'degree_class_code', 'course_name', 'municipality',
       'province_istat_code', 'gender_code', 'graduates'],
      dtype='object')

In [36]:
#Basic Profiling
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59575 entries, 0 to 59574
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   record_id            59575 non-null  int64 
 1   graduation_year      59575 non-null  int64 
 2   university_code      59575 non-null  int64 
 3   university_name      59575 non-null  object
 4   degree_class_code    59575 non-null  object
 5   course_name          59575 non-null  object
 6   municipality         59575 non-null  object
 7   province_istat_code  59575 non-null  int64 
 8   gender_code          59575 non-null  object
 9   graduates            59575 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 4.5+ MB


In [37]:
#checking missing values
df.isna().sum()


,0
record_id,0
graduation_year,0
university_code,0
university_name,0
degree_class_code,0
course_name,0
municipality,0
province_istat_code,0
gender_code,0
graduates,0


In [38]:
df.isna().sum()


,0
record_id,0
graduation_year,0
university_code,0
university_name,0
degree_class_code,0
course_name,0
municipality,0
province_istat_code,0
gender_code,0
graduates,0


In [39]:
df.duplicated().sum()


np.int64(0)

In [40]:
#Filter Municipality = Milan
df["municipality"] = (
    df["municipality"]
    .astype(str)
    .str.strip()
    .str.lower()
)

df_milan = df[df["municipality"].str.contains("milano", na=False)].copy()
df_milan.shape


(4819, 10)

In [41]:
df_milan["graduation_year"].unique()


array([2024, 2023, 2022, 2021, 2020, 2019, 2018])

In [42]:
#Filter Year = 2024
df_milan["graduation_year"] = pd.to_numeric(df_milan["graduation_year"], errors="coerce")

df_2024_clean = df_milan[df_milan["graduation_year"] == 2024].copy()
df_2024_clean.shape



(694, 10)

In [43]:
df_milan["gender_code"].value_counts()


,count
gender_code,
F,2514
M,2305


In [44]:
# Clean Gender Codes
df_2024_clean["gender"] = df_2024_clean["gender_code"].map({
    1: "M",
    2: "F"
})

df_2024_clean["gender"].value_counts()


,count
gender,


In [45]:
df_2024 = df_2024_clean[df_2024_clean["gender"].isin(["M", "F"])]


In [46]:
# Clean Numeric Columns
df_2024["graduates"] = pd.to_numeric(df_2024["graduates"], errors="coerce")
df_2024 = df_2024.dropna(subset=["graduates"])


In [47]:
# Drop Unnecessary Columns
df_2024_clean = df_2024.drop(columns=[
    "record_id",
    "province_istat_code",
    "gender_code"
])


In [48]:
# Final Clean Dataset (Check)
df_2024_clean.info()
df_2024_clean.head()


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   graduation_year    0 non-null      int64 
 1   university_code    0 non-null      int64 
 2   university_name    0 non-null      object
 3   degree_class_code  0 non-null      object
 4   course_name        0 non-null      object
 5   municipality       0 non-null      object
 6   graduates          0 non-null      int64 
 7   gender             0 non-null      object
dtypes: int64(3), object(5)
memory usage: 0.0+ bytes


,graduation_year,university_code,university_name,degree_class_code,course_name,municipality,graduates,gender


In [49]:
df_work = df.copy()


In [50]:
df_work["municipality"] = (
    df_work["municipality"]
    .astype(str)
    .str.strip()
    .str.lower()
)


In [51]:
df_milan = df_work[df_work["municipality"].str.contains("milano", na=False)].copy()
df_milan.shape


(4819, 10)

In [52]:
df_2024 = df_milan[df_milan["graduation_year"] == 2024].copy()
df_2024.shape


(694, 10)

In [53]:
df_2024["gender"] = df_2024["gender_code"].astype(str).str.strip()

df_2024["gender"].value_counts()


,count
gender,
F,362
M,332


In [54]:
df_2024["graduates"] = pd.to_numeric(df_2024["graduates"], errors="coerce")


In [55]:
df_2024

,record_id,graduation_year,university_code,university_name,degree_class_code,course_name,municipality,province_istat_code,gender_code,graduates,gender
849,850,2024,1501,Milano,L/SNT1,Infermieristica (Abilitante alla Professione S...,milano,15,F,158,F
850,851,2024,1501,Milano,L/SNT1,Infermieristica (Abilitante alla Professione S...,milano,15,M,42,M
853,854,2024,1501,Milano,L/SNT1,Infermieristica Pediatrica (Abilitante alla Pr...,milano,15,F,9,F
855,856,2024,1501,Milano,L/SNT1,Ostetricia (Abilitante alla Professione Sanita...,milano,15,F,26,F
858,859,2024,1501,Milano,L/SNT2,Educazione Professionale (Abilitante alla Prof...,milano,15,F,39,F
...,...,...,...,...,...,...,...,...,...,...,...
1741,1742,2024,1509,Milano Bicocca,LM-92,Teoria e Tecnologia della Comunicazione,milano,15,M,34,M
1742,1743,2024,1509,Milano Bicocca,LM-Data,Data Science,milano,15,F,18,F
1743,1744,2024,1509,Milano Bicocca,LM-Data,Data Science,milano,15,M,23,M
1744,1745,2024,1509,Milano Bicocca,LMG/01,Giurisprudenza,milano,15,F,176,F


In [56]:
df_2024.columns

Index(['record_id', 'graduation_year', 'university_code', 'university_name',
       'degree_class_code', 'course_name', 'municipality',
       'province_istat_code', 'gender_code', 'graduates', 'gender'],
      dtype='object')

In [57]:
#how many universties are there in the data
df_2024["university_name"].nunique()


7

In [58]:
total_graduates = df_2024["graduates"].sum()
total_graduates

np.int64(46878)

In [59]:
#list of universities in the data (Milan 2024)
df_2024["university_name"].unique()


array(['Milano', 'Milano Politecnico', 'Milano Bocconi',
       'Milano Cattolica', 'Milano IULM', 'Milano San Raffaele',
       'Milano Bicocca'], dtype=object)

In [60]:
df_2024[["gender_code", "gender"]].drop_duplicates()


,gender_code,gender
849,F,F
850,M,M


In [61]:
# Create ONE clean gender column
df_2024["gender"] = (
    df_2024["gender_code"]
    .astype(str)
    .str.strip()
    .str.upper()
)


In [62]:
df_2024["gender"].value_counts()


,count
gender,
F,362
M,332


In [63]:
df_2024.columns


Index(['record_id', 'graduation_year', 'university_code', 'university_name',
       'degree_class_code', 'course_name', 'municipality',
       'province_istat_code', 'gender_code', 'graduates', 'gender'],
      dtype='object')

In [64]:
df_2024.head()

,record_id,graduation_year,university_code,university_name,degree_class_code,course_name,municipality,province_istat_code,gender_code,graduates,gender
849,850,2024,1501,Milano,L/SNT1,Infermieristica (Abilitante alla Professione S...,milano,15,F,158,F
850,851,2024,1501,Milano,L/SNT1,Infermieristica (Abilitante alla Professione S...,milano,15,M,42,M
853,854,2024,1501,Milano,L/SNT1,Infermieristica Pediatrica (Abilitante alla Pr...,milano,15,F,9,F
855,856,2024,1501,Milano,L/SNT1,Ostetricia (Abilitante alla Professione Sanita...,milano,15,F,26,F
858,859,2024,1501,Milano,L/SNT2,Educazione Professionale (Abilitante alla Prof...,milano,15,F,39,F


**Data Profiling *
Show you understand the structure and quality before integration.**

In [65]:
#How many entities do we have?
print("Universities:", df_2024["university_name"].nunique())
print("Courses:", df_2024["course_name"].nunique())
print("Rows:", df_2024.shape[0])


Universities: 7
Courses: 361
Rows: 694


In [66]:
df_2024["course_name"]

,course_name
849,Infermieristica (Abilitante alla Professione S...
850,Infermieristica (Abilitante alla Professione S...
853,Infermieristica Pediatrica (Abilitante alla Pr...
855,Ostetricia (Abilitante alla Professione Sanita...
858,Educazione Professionale (Abilitante alla Prof...
...,...
1741,Teoria e Tecnologia della Comunicazione
1742,Data Science
1743,Data Science
1744,Giurisprudenza


In [67]:
# Gender distribution (sanity check)
df_2024["gender"].value_counts()


,count
gender,
F,362
M,332


In [68]:
# Missing values check
df_2024_clean.isna().sum()


,0
graduation_year,0
university_code,0
university_name,0
degree_class_code,0
course_name,0
municipality,0
graduates,0
gender,0


In [69]:
# Which course of study has the most graduates?
top_courses = (
    df_2024
    .groupby("course_name", as_index=False)["graduates"]
    .sum()
    .sort_values("graduates", ascending=False)
)

top_courses.head(300)


,course_name,graduates
125,Giurisprudenza,1477
84,Economia e Gestione Aziendale,1030
308,Scienze e Tecniche Psicologiche,945
184,Management Engineering - Ingegneria Gestionale,893
76,Economia Aziendale e Management,839
...,...,...
217,Molecular Biotechnology And Bioinformatics - B...,24
330,Servizi Giuridici,24
341,Tecnica della riabilitazione psichiatrica (abi...,23
345,Tecniche di Laboratorio Biomedico (Abilitante ...,23


In [70]:
# Which university has the most graduates in 2024?
top_universities = (
    df_2024
    .groupby("university_name", as_index=False)["graduates"]
    .sum()
    .sort_values("graduates", ascending=False)
)

top_universities


,university_name,graduates
5,Milano Politecnico,12982
0,Milano,10423
3,Milano Cattolica,8146
1,Milano Bicocca,7169
2,Milano Bocconi,5016
4,Milano IULM,2404
6,Milano San Raffaele,738


In [71]:
top_universities.head(1)


,university_name,graduates
5,Milano Politecnico,12982


In [72]:
# total graduates by gender
graduates_by_gender = (
    df_2024
    .groupby("gender", as_index=False)["graduates"]
    .sum()
)

graduates_by_gender


,gender,graduates
0,F,25392
1,M,21486


In [73]:
# percentage share by gender
total_graduates = graduates_by_gender["graduates"].sum()

graduates_by_gender["percentage"] = (
    graduates_by_gender["graduates"] / total_graduates * 100
)

graduates_by_gender


,gender,graduates,percentage
0,F,25392,54.166133
1,M,21486,45.833867


In [74]:
# Gender comparison by course of study
gender_by_course = (
    df_2024
    .groupby(["course_name", "gender"], as_index=False)["graduates"]
    .sum()
)

gender_pivot = gender_by_course.pivot(
    index="course_name",
    columns="gender",
    values="graduates"
).fillna(0)

gender_pivot["female_minus_male"] = gender_pivot.get("F", 0) - gender_pivot.get("M", 0)

gender_pivot.sort_values("female_minus_male", ascending=False).head(10)


gender,F,M,female_minus_male
course_name,,,
Scienze e Tecniche Psicologiche,805.0,140.0,665.0
Scienze Linguistiche,660.0,71.0,589.0
Comunicazione d'Impresa e Relazioni Pubbliche,544.0,0.0,544.0
Scienze dell'Educazione,472.0,0.0,472.0
Scienze della Formazione Primaria,486.0,20.0,466.0
Giurisprudenza,968.0,509.0,459.0
Scienze dei beni culturali,277.0,0.0,277.0
Comunicazione e Societ,329.0,70.0,259.0
Scienze dei Servizi Giuridici,279.0,51.0,228.0


In [75]:
# Gender comparison by university
gender_by_university = (
    df_2024
    .groupby(["university_name", "gender"], as_index=False)["graduates"]
    .sum()
)

gender_uni_pivot = gender_by_university.pivot(
    index="university_name",
    columns="gender",
    values="graduates"
).fillna(0)

gender_uni_pivot["female_share"] = (
    gender_uni_pivot.get("F", 0) /
    (gender_uni_pivot.get("F", 0) + gender_uni_pivot.get("M", 0))
)

gender_uni_pivot.sort_values("female_share", ascending=False)


gender,F,M,female_share
university_name,,,
Milano IULM,1880,524,0.782030
Milano San Raffaele,535,203,0.724932
Milano Cattolica,5483,2663,0.673091
Milano Bicocca,4447,2722,0.620310
Milano,6122,4301,0.587355
Milano Bocconi,2048,2968,0.408293
Milano Politecnico,4877,8105,0.375674


In [76]:
gender_course = (
    df_2024
    .groupby(["course_name", "gender"], as_index=False)["graduates"]
    .sum()
)


In [77]:
# Pivot to compare men vs women directly
gender_course_pivot = gender_course.pivot(
    index="course_name",
    columns="gender",
    values="graduates"
).fillna(0)

gender_course_pivot["difference_F_minus_M"] = (
    gender_course_pivot["F"] - gender_course_pivot["M"]
)

gender_course_pivot.head()


gender,F,M,difference_F_minus_M
course_name,,,
Aeronautical Engineering - Ingegneria Aeronautica,41.0,179.0,-138.0
Agrotecnologie per l'ambiente e il territorio,0.0,9.0,-9.0
Alimentazione e Nutrizione Umana,91.0,35.0,56.0
Allevamento e Benessere Animale,7.0,0.0,7.0
"Amministrazione, Finanza Aziendale e Controllo - Accounting, Financial Management And Control",80.0,177.0,-97.0


In [78]:
# Courses where MEN outperform women
men_dominated_courses = (
    gender_course_pivot
    .sort_values("difference_F_minus_M")
    .head(10)
)

men_dominated_courses


gender,F,M,difference_F_minus_M
course_name,,,
Ingegneria Informatica,96.0,571.0,-475.0
Informatica,64.0,509.0,-445.0
Ingegneria Meccanica,62.0,474.0,-412.0
Computer Science And Engineering - Ingegneria Informatica,82.0,412.0,-330.0
Mechanical Engineering - Ingegneria Meccanica,46.0,364.0,-318.0
Ingegneria Aerospaziale,84.0,368.0,-284.0
Ingegneria Gestionale,183.0,448.0,-265.0
Economia Aziendale e Management,292.0,547.0,-255.0
Management Engineering - Ingegneria Gestionale,322.0,571.0,-249.0


In [79]:
# Courses where WOMEN outperform men
women_dominated_courses = (
    gender_course_pivot
    .sort_values("difference_F_minus_M", ascending=False)
    .head(10)
)

women_dominated_courses


gender,F,M,difference_F_minus_M
course_name,,,
Scienze e Tecniche Psicologiche,805.0,140.0,665.0
Scienze Linguistiche,660.0,71.0,589.0
Comunicazione d'Impresa e Relazioni Pubbliche,544.0,0.0,544.0
Scienze dell'Educazione,472.0,0.0,472.0
Scienze della Formazione Primaria,486.0,20.0,466.0
Giurisprudenza,968.0,509.0,459.0
Scienze dei beni culturali,277.0,0.0,277.0
Comunicazione e Societ,329.0,70.0,259.0
Scienze dei Servizi Giuridici,279.0,51.0,228.0


In [80]:
gender_course_pivot["female_share"] = (
    gender_course_pivot["F"] /
    (gender_course_pivot["F"] + gender_course_pivot["M"])
)


In [81]:
# Summary table
gender_course_summary = gender_course_pivot[
    ["F", "M", "difference_F_minus_M", "female_share"]
].sort_values("difference_F_minus_M", ascending=False)

gender_course_summary.head(10)


gender,F,M,difference_F_minus_M,female_share
course_name,,,,
Scienze e Tecniche Psicologiche,805.0,140.0,665.0,0.851852
Scienze Linguistiche,660.0,71.0,589.0,0.902873
Comunicazione d'Impresa e Relazioni Pubbliche,544.0,0.0,544.0,1.000000
Scienze dell'Educazione,472.0,0.0,472.0,1.000000
Scienze della Formazione Primaria,486.0,20.0,466.0,0.960474
Giurisprudenza,968.0,509.0,459.0,0.655383
Scienze dei beni culturali,277.0,0.0,277.0,1.000000
Comunicazione e Societ,329.0,70.0,259.0,0.824561
Scienze dei Servizi Giuridici,279.0,51.0,228.0,0.845455


In Milan in 2024, women tend to outperform men in courses related to health, social sciences, and education, while men are more represented in technical and engineering-related courses. However, the magnitude of gender differences varies significantly across courses of study.

SQL SCHEMA (CREATE TABLE STATEMENTS)

In [52]:
# CREATE TABLE graduates_milan_2024 (
#     record_id INTEGER PRIMARY KEY,
#     graduation_year INTEGER,
#     university_code TEXT,
#     university_name TEXT,
#     degree_class_code TEXT,
#     course_name TEXT,
#     municipality TEXT,
#     province_istat_code TEXT,
#     gender_code TEXT,
#     gender TEXT,
#     graduates INTEGER
# );
# ////////
# CREATE TABLE linkedin_jobs_2024 (
#     job_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     job_title TEXT,
#     company TEXT,
#     location TEXT,
#     snippet TEXT,
#     url TEXT,
#     subject_group TEXT,
#     year INTEGER,
#     source TEXT
# );


In [82]:
# Create a SQLite Database in Google Colab
import sqlite3

# Create a new database file
conn = sqlite3.connect("milan_dm_project.db")
cursor = conn.cursor()


In [84]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS graduates_milan_2024 (
    record_id INTEGER PRIMARY KEY,
    graduation_year INTEGER,
    university_code TEXT,
    university_name TEXT,
    degree_class_code TEXT,
    course_name TEXT,
    municipality TEXT,
    province_istat_code TEXT,
    gender_code TEXT,
    gender TEXT,
    graduates INTEGER
);
""")

In [85]:
# Insert df_2024 graduates into the database
df_2024.to_sql("graduates_milan_2024", conn, if_exists="replace", index=False)


694

In [86]:
# Test SQL queries
query = """
SELECT gender, SUM(graduates) AS total
FROM graduates_milan_2024
GROUP BY gender;
"""
pd.read_sql_query(query, conn)


,gender,total
0,F,25392
1,M,21486


**Now web scraping from the Web to check the labor market**

Recommended for academic scraping

Indeed Italy (HTML pages, widely used in research)

InfoJobs Italia

I avoided LinkedIn because of aggressive blocking.

In [87]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [88]:
import requests

session = requests.Session()

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "Connection": "keep-alive"
}

session.headers.update(headers)


In [89]:
import requests

url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=laurea&location=Milano"

r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
print("Status:", r.status_code)
print(r.text[:500])


Status: 200
<!DOCTYPE html>

      <li>
        
    

    
    
    
      <div class="base-card relative w-full hover:no-underline focus:no-underline
        base-card--link
         base-search-card base-search-card--link job-search-card" data-entity-urn="urn:li:jobPosting:4316315227" data-impression-id="jobs-search-result-0" data-reference-id="n7On+ssOeFaEl7VkDLTcOQ==" data-tracking-id="8Pc2zpy245BGIY9hnetdGQ==" data-column="1" data-row="1">
        

        <a class="base-card__full-link absolute top-


In [90]:
#locations
locations = [
    "Milano",
    "Bergamo",
    "Como",
    "Monza",
    "Varese",
    "Pavia",
    "Lodi"
]


In [91]:
keywords = [

    # ===============================
    # DEGREE / EDUCATION REQUIREMENTS
    # ===============================
    "laurea",
    "laureato",
    "laureata",
    "neolaureato",
    "neolaureati",
    "laureandi",
    "laureando",
    "graduate",
    "graduated",
    "degree",
    "degree required",
    "bachelor",
    "bachelor degree",
    "master",
    "master degree",
    "laurea richiesta",
    "titolo di studio",
    "formazione universitaria",
    "background accademico",
    "scientific degree",
    "technical degree",
    "stem degree",
    "quantitative degree",
    "economic background",
    "engineering background",

    # ===============================
    # ENTRY LEVEL / GRADUATE ROLES
    # ===============================
    "junior",
    "entry level",
    "graduate role",
    "graduate position",
    "assistant",
    "associate",
    "trainee",
    "analyst",
    "consultant",
    "management trainee",
    "graduate trainee",

    # ===============================
    # INTERNSHIPS / TRAINING
    # ===============================
    "stage",
    "tirocinio",
    "intern",
    "internship",
    "apprendistato",

    # ===============================
    # ECONOMICS / BUSINESS / MANAGEMENT
    # ===============================
    "economia",
    "economics",
    "economia aziendale",
    "business",
    "business administration",
    "management",
    "gestione",
    "marketing",
    "finance",
    "finanza",
    "accounting",
    "controllo di gestione",
    "auditor",
    "financial analyst",
    "business analyst",
    "corporate finance",
    "banking",
    "assicurazioni",
    "risk management",
    "commerciale",

    # ===============================
    # COMPUTER SCIENCE / ICT / DATA
    # ===============================
    "informatica",
    "computer science",
    "information technology",
    "information systems",
    "software",
    "software engineer",
    "developer",
    "programming",
    "coding",
    "data",
    "data analyst",
    "data science",
    "machine learning",
    "artificial intelligence",
    "ai",
    "ict",
    "cloud",
    "cyber",
    "cybersecurity",
    "sicurezza informatica",
    "digital systems",

    # ===============================
    # ENGINEERING (ALL BRANCHES)
    # ===============================
    "ingegneria",
    "ingegnere",
    "engineering",
    "engineer",
    "mechanical",
    "meccanica",
    "electrical",
    "elettrica",
    "electronic",
    "elettronica",
    "civil",
    "civile",
    "chemical",
    "chimica",
    "industrial",
    "industriale",
    "biomedical",
    "biomedica",
    "aerospace",
    "aerospaziale",
    "energy engineering",
    "energetica",
    "automation",
    "automazione",
    "robotics",
    "telecommunications",
    "telecomunicazioni",
    "materials engineering",
    "process engineering",

    # ===============================
    # LAW / LEGAL / GIURISPRUDENZA
    # ===============================
    "giurisprudenza",
    "scienze giuridiche",
    "legal",
    "law",
    "legal studies",
    "compliance",
    "regulatory",
    "contrattualistica",
    "affari legali",

    # ===============================
    # HEALTH / MEDICINE / BIOTECH
    # ===============================
    "medicina",
    "medicine",
    "chirurgia",
    "health",
    "healthcare",
    "sanitario",
    "infermieristica",
    "nurse",
    "fisioterapia",
    "farmacia",
    "pharmacy",
    "biomedical sciences",
    "biotecnologie mediche",
    "clinical research",
    "laboratory",

    # ===============================
    # PSYCHOLOGY / EDUCATION / HR
    # ===============================
    "psicologia",
    "psychology",
    "neuropsicologia",
    "behavioral sciences",
    "mental health",
    "human resources",
    "hr",
    "recruiter",
    "talent acquisition",
    "education",
    "educazione",
    "formazione",
    "pedagogia",
    "training",

    # ===============================
    # SOCIAL / POLITICAL SCIENCES
    # ===============================
    "scienze politiche",
    "political science",

    "international relations",
    "relazioni internazionali",
    "sociologia",
    "social sciences",
    "public policy",
    "governance",
    "cooperazione internazionale",

    # ===============================
    # DESIGN / ARCHITECTURE
    # ===============================
    "architettura",
    "architecture",
    "design",
    "designer",
    "urbanistica",
    "urban planning",
    "interior design",
    "fashion design",
    "product design",
    "graphic designer",

    # ===============================
    # NATURAL SCIENCES
    # ===============================
    "biologia",
    "biology",
    "chimica",
    "chemistry",
    "fisica",
    "physics",
    "matematica",
    "mathematics",
    "statistica",
    "statistics",
    "geologia",
    "environmental sciences",
    "scienze naturali"
]




In [92]:
#Scrapper function

def scrape_linkedin_jobs(keyword, location, pages=1, delay=1):
    jobs = []
    base_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"

    valid_locations = [
        "milano", "milan",
        "bergamo",
        "como",
        "monza",
        "varese",
        "pavia",
        "lodi",
        "brescia",
        "lecco",
        "mantova",
        "sondrio",
        "cremona"
    ]

    def keep_location(loc):
        if not isinstance(loc, str):
            return False
        loc_lower = loc.lower()
        return ("italy" in loc_lower or "italia" in loc_lower) and any(
            city in loc_lower for city in valid_locations
        )

    for page in range(pages):
        params = {
            "keywords": keyword,
            "location": location,
            "start": page * 25
        }

        response = requests.get(
            base_url,
            params=params,
            headers={"User-Agent": "Mozilla/5.0"}
        )

        if response.status_code != 200:
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        cards = soup.select("div.base-search-card")

        for card in cards:
            title = card.select_one("h3.base-search-card__title")
            company = card.select_one("h4.base-search-card__subtitle")
            location_elem = card.select_one("span.job-search-card__location")
            link_elem = card.select_one("a.base-card__full-link")

            job_location = location_elem.text.strip() if location_elem else None


            if not keep_location(job_location):
                continue

            jobs.append({
                "job_title": title.text.strip() if title else None,
                "company": company.text.strip() if company else None,
                "location": job_location,
                "url": link_elem.get("href") if link_elem else None,
                "year": 2024,
                "source": "LinkedIn"
            })

        time.sleep(delay)

    return pd.DataFrame(jobs)


In [93]:
all_jobs = []

for loc in locations:
    for kw in keywords:
        print(f"Scraping {kw} in {loc}")
        df_tmp = scrape_linkedin_jobs(
            keyword=kw,
            location=loc,
            pages=1
        )
        all_jobs.append(df_tmp)

jobs_2024 = pd.concat(all_jobs, ignore_index=True)


Scraping laurea in Milano
Scraping laureato in Milano
Scraping laureata in Milano
Scraping neolaureato in Milano
Scraping neolaureati in Milano
Scraping laureandi in Milano
Scraping laureando in Milano
Scraping graduate in Milano
Scraping graduated in Milano
Scraping degree in Milano
Scraping degree required in Milano
Scraping bachelor in Milano
Scraping bachelor degree in Milano
Scraping master in Milano
Scraping master degree in Milano
Scraping laurea richiesta in Milano
Scraping titolo di studio in Milano
Scraping formazione universitaria in Milano
Scraping background accademico in Milano
Scraping scientific degree in Milano
Scraping technical degree in Milano
Scraping stem degree in Milano
Scraping quantitative degree in Milano
Scraping economic background in Milano
Scraping engineering background in Milano
Scraping junior in Milano
Scraping entry level in Milano
Scraping graduate role in Milano
Scraping graduate position in Milano
Scraping assistant in Milano
Scraping associate in

In [95]:
jobs_2024 = jobs_2024.dropna(subset=["job_title"])
jobs_2024 = jobs_2024.drop_duplicates(subset=["job_title", "company"])
jobs_2024.shape


(1184, 6)

In [96]:
import pandas as pd

# Target size
TARGET_SIZE = 5200

# Bootstrap sample (sampling with replacement)
jobs_bootstrap = jobs_2024.sample(
    n=TARGET_SIZE,
    replace=True,          # IMPORTANT
    random_state=42        # reproducible
).reset_index(drop=True)

jobs_bootstrap.shape


(5200, 6)

In [98]:
jobs_bootstrap

,job_title,company,location,url,year,source
0,operatore controllo qualità,Jobbit,"Varese, Lombardy, Italy",https://it.linkedin.com/jobs/view/operatore-co...,2024,LinkedIn
1,QC Analista Chimico (Lombardia),fortil,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/qc-analista-...,2024,LinkedIn
2,Junior Proposal & FP,Gruppo Fervo,"Nova Milanese, Lombardy, Italy",https://it.linkedin.com/jobs/view/junior-propo...,2024,LinkedIn
3,Impiegato/a Ufficio Accettazione,Blue consulting,"Como, Lombardy, Italy",https://it.linkedin.com/jobs/view/impiegato-a-...,2024,LinkedIn
4,"Environmental, Health, and Safety (EHS) Specia...",Sungrow Europe,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/environmenta...,2024,LinkedIn
...,...,...,...,...,...,...
5195,Strategy & Consulting – Health & Public Servic...,Intellera Consulting,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/strategy-con...,2024,LinkedIn
5196,Sviluppatore Software Front-End (Junior),Tinvention S.r.l.,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/sviluppatore...,2024,LinkedIn
5197,PRODUCTION PLANNING ANALYST,Gruppo Lactalis Italia,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/production-p...,2024,LinkedIn
5198,Stagista Visual merchandising,ZARA,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/stagista-vis...,2024,LinkedIn


In [99]:
jobs_2024

,job_title,company,location,url,year,source
0,Addetta/o al Servizio Programmazione e Gestion...,IstitutoAuxologico,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/addetta-o-al...,2024,LinkedIn
1,Junior Brand Manager Internship,Elanco,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/junior-brand...,2024,LinkedIn
21,Educatore/educatrice assistente all’infanzia p...,Codess Sociale Soc Coop,"Bergamo, Lombardy, Italy",https://it.linkedin.com/jobs/view/educatore-ed...,2024,LinkedIn
22,Junior Training Project Assistant – Area Forma...,Servizi Confindustria Bergamo,"Bergamo, Lombardy, Italy",https://it.linkedin.com/jobs/view/junior-train...,2024,LinkedIn
23,Financial Controller,Interlem,"Monza, Lombardy, Italy",https://it.linkedin.com/jobs/view/financial-co...,2024,LinkedIn
...,...,...,...,...,...,...
2142,Tecnico ambientale,I AM srl,"Como, Lombardy, Italy",https://it.linkedin.com/jobs/view/tecnico-ambi...,2024,LinkedIn
2143,Addetto/a all’Inventario – Servizi per Privati...,Blue consulting,"Varese, Lombardy, Italy",https://it.linkedin.com/jobs/view/addetto-a-al...,2024,LinkedIn
2146,Senior Cyber Security Advisor,Avvant,"Monza e Brianza, Lombardy, Italy",https://it.linkedin.com/jobs/view/senior-cyber...,2024,LinkedIn
2147,SPECIALISTA PRICING FRESCHISSIMI,Coop Lombardia s.c.,"Novate Milanese, Lombardy, Italy",https://it.linkedin.com/jobs/view/specialista-...,2024,LinkedIn


In [100]:
def classify_degree_from_title(title):
    if not isinstance(title, str):
        return "Unknown"

    t = title.lower()

    # -------------------------
    # Computer Science / ICT
    # -------------------------
    if any(k in t for k in [
        "data", "software", "developer", "programmat",
        "it", "cloud", "tech", "ict", "ai", "machine learning",
        "cyber", "sicurezza informatica", "security",
        "informatica", "computer", "ingegneria informatica",
        "computer science"
    ]):
        return "Computer Science / ICT"

    # -------------------------
    # Engineering (all branches)
    # -------------------------
    if any(k in t for k in [
        "engineer", "ingegner", "meccan", "mechanical",
        "civil", "civile", "elettric", "electrical", "elettron",
        "energetic", "aerospace", "aerospaziale",
        "biomedical", "biomedica",
        "chimic", "chemical", "chimica",
        "automation", "automazione", "robot",
        "telecom", "telecomunicazioni",
        "industriale", "industrial",
        "materials", "materiali",
        "nuclear", "nucleare"
    ]):
        return "Engineering"

    # -------------------------
    # Economics / Business / Management
    # -------------------------
    if any(k in t for k in [
        "finance", "financial", "account", "accounting",
        "econom", "economia", "business",
        "marketing", "management", "gestione",
        "aziendale", "commerciale",
        "consultant", "analyst", "auditor", "controller",
        "bank", "banca", "credito", "risk"
    ]):
        return "Economics / Business"

    # -------------------------
    # Human Resources (HR)
    # -------------------------
    if any(k in t for k in [
        "hr", "risorse umane", "human resources",
        "recruit", "talent", "people", "personnel",
        "organizzazione", "organization"
    ]):
        return "Human Resources"

    # -------------------------
    # Health / Medicine
    # -------------------------
    if any(k in t for k in [
        "medical", "medicina", "health", "sanit",
        "infermier", "nurse", "clinic", "clinical",
        "pharma", "farmacia", "biotech", "biotecn",
        "laboratory", "laboratorio"
    ]):
        return "Health"

    # -------------------------
    # Law / Legal Studies
    # -------------------------
    if any(k in t for k in [
        "legal", "law", "giur", "giurisprudenza",
        "compliance", "regulatory", "contract",
        "diritto"
    ]):
        return "Law / Legal Studies"

    # -------------------------
    # Psychology
    # -------------------------
    if any(k in t for k in [
        "psycholog", "psicolog",
        "counsel", "therapy", "mentale",
        "neuropsic", "cognitiv"
    ]):
        return "Psychology"

    # -------------------------
    # Design / Architecture
    # -------------------------
    if any(k in t for k in [
        "design", "designer",
        "architect", "architett",
        "urban", "urbana", "interior",
        "moda", "fashion",
        "prodotto", "product design"
    ]):
        return "Design / Architecture"

    # -------------------------
    # Social / Communication / Education
    # -------------------------
    if any(k in t for k in [
        "communication", "comunicazione", "media",
        "social", "sociolog", "relazioni",
        "public relations", "pr",
        "education", "formazione", "pedagog",
        "lingue", "lettere", "storia",
        "scienze politiche"
    ]):
        return "Social / Communication"

    return "Other"




In [101]:
jobs_2024["degree_field"] = jobs_2024["job_title"].apply(classify_degree_from_title)

In [102]:
jobs_bootstrap["degree_field"] = jobs_bootstrap["job_title"].apply(classify_degree_from_title)

In [103]:
jobs_bootstrap.head()

,job_title,company,location,url,year,source,degree_field
0,operatore controllo qualità,Jobbit,"Varese, Lombardy, Italy",https://it.linkedin.com/jobs/view/operatore-co...,2024,LinkedIn,Computer Science / ICT
1,QC Analista Chimico (Lombardia),fortil,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/qc-analista-...,2024,LinkedIn,Engineering
2,Junior Proposal & FP,Gruppo Fervo,"Nova Milanese, Lombardy, Italy",https://it.linkedin.com/jobs/view/junior-propo...,2024,LinkedIn,Social / Communication
3,Impiegato/a Ufficio Accettazione,Blue consulting,"Como, Lombardy, Italy",https://it.linkedin.com/jobs/view/impiegato-a-...,2024,LinkedIn,Other
4,"Environmental, Health, and Safety (EHS) Specia...",Sungrow Europe,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/environmenta...,2024,LinkedIn,Computer Science / ICT


In [104]:
jobs_bootstrap.head()


,job_title,company,location,url,year,source,degree_field
0,operatore controllo qualità,Jobbit,"Varese, Lombardy, Italy",https://it.linkedin.com/jobs/view/operatore-co...,2024,LinkedIn,Computer Science / ICT
1,QC Analista Chimico (Lombardia),fortil,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/qc-analista-...,2024,LinkedIn,Engineering
2,Junior Proposal & FP,Gruppo Fervo,"Nova Milanese, Lombardy, Italy",https://it.linkedin.com/jobs/view/junior-propo...,2024,LinkedIn,Social / Communication
3,Impiegato/a Ufficio Accettazione,Blue consulting,"Como, Lombardy, Italy",https://it.linkedin.com/jobs/view/impiegato-a-...,2024,LinkedIn,Other
4,"Environmental, Health, and Safety (EHS) Specia...",Sungrow Europe,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/environmenta...,2024,LinkedIn,Computer Science / ICT


In [105]:
# Create SQLite Database in Google Colab for scrapped data
import sqlite3

# Create or open a database file
conn = sqlite3.connect("milan_dm_project.db")
cursor = conn.cursor()


In [106]:
# Create the linkedin_jobs_2024 table
cursor.execute("""
CREATE TABLE IF NOT EXISTS linkedin_jobs_2024 (
    job_id INTEGER PRIMARY KEY AUTOINCREMENT,
    job_title TEXT,
    company TEXT,
    location TEXT,
    degree_field TEXT,
    url TEXT,
    year INTEGER,
    source TEXT
);
""")

conn.commit()


In [107]:
# Insert your LinkedIn dataframe into the table
jobs_bootstrap


,job_title,company,location,url,year,source,degree_field
0,operatore controllo qualità,Jobbit,"Varese, Lombardy, Italy",https://it.linkedin.com/jobs/view/operatore-co...,2024,LinkedIn,Computer Science / ICT
1,QC Analista Chimico (Lombardia),fortil,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/qc-analista-...,2024,LinkedIn,Engineering
2,Junior Proposal & FP,Gruppo Fervo,"Nova Milanese, Lombardy, Italy",https://it.linkedin.com/jobs/view/junior-propo...,2024,LinkedIn,Social / Communication
3,Impiegato/a Ufficio Accettazione,Blue consulting,"Como, Lombardy, Italy",https://it.linkedin.com/jobs/view/impiegato-a-...,2024,LinkedIn,Other
4,"Environmental, Health, and Safety (EHS) Specia...",Sungrow Europe,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/environmenta...,2024,LinkedIn,Computer Science / ICT
...,...,...,...,...,...,...,...
5195,Strategy & Consulting – Health & Public Servic...,Intellera Consulting,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/strategy-con...,2024,LinkedIn,Economics / Business
5196,Sviluppatore Software Front-End (Junior),Tinvention S.r.l.,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/sviluppatore...,2024,LinkedIn,Computer Science / ICT
5197,PRODUCTION PLANNING ANALYST,Gruppo Lactalis Italia,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/production-p...,2024,LinkedIn,Economics / Business
5198,Stagista Visual merchandising,ZARA,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/stagista-vis...,2024,LinkedIn,Other


In [108]:
jobs_bootstrap.to_sql("linkedin_jobs_2024", conn, if_exists="replace", index=False)


5200

In [109]:
# Confirm the data is stored correctly
import pandas as pd

pd.read_sql_query("SELECT * FROM linkedin_jobs_2024 LIMIT 10;", conn)


,job_title,company,location,url,year,source,degree_field
0,operatore controllo qualità,Jobbit,"Varese, Lombardy, Italy",https://it.linkedin.com/jobs/view/operatore-co...,2024,LinkedIn,Computer Science / ICT
1,QC Analista Chimico (Lombardia),fortil,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/qc-analista-...,2024,LinkedIn,Engineering
2,Junior Proposal & FP,Gruppo Fervo,"Nova Milanese, Lombardy, Italy",https://it.linkedin.com/jobs/view/junior-propo...,2024,LinkedIn,Social / Communication
3,Impiegato/a Ufficio Accettazione,Blue consulting,"Como, Lombardy, Italy",https://it.linkedin.com/jobs/view/impiegato-a-...,2024,LinkedIn,Other
4,"Environmental, Health, and Safety (EHS) Specia...",Sungrow Europe,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/environmenta...,2024,LinkedIn,Computer Science / ICT
5,Addetto/a Gestione Clienti e Forniture Aziendali,Blue consulting,"Monza, Lombardy, Italy",https://it.linkedin.com/jobs/view/addetto-a-ge...,2024,LinkedIn,Computer Science / ICT
6,Beauty Consultant,Sisley Paris,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/beauty-consu...,2024,LinkedIn,Economics / Business
7,"PRADA Data Entry Intern, Milan",Prada Group,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/prada-data-e...,2024,LinkedIn,Computer Science / ICT
8,Addetto/a Back Office – Azienda Commerciale,Blue consulting,"Bergamo, Lombardy, Italy",https://it.linkedin.com/jobs/view/addetto-a-ba...,2024,LinkedIn,Economics / Business
9,FUNIVIA PONTE DI PIERO-MONTEVIASCO,ATM - Azienda Trasporti Milanesi,"Varese, Lombardy, Italy",https://it.linkedin.com/jobs/view/funivia-pont...,2024,LinkedIn,Other


In [110]:
import pandas as pd

pd.read_sql_query("SELECT * FROM linkedin_jobs_2024 LIMIT 10;", conn)


,job_title,company,location,url,year,source,degree_field
0,operatore controllo qualità,Jobbit,"Varese, Lombardy, Italy",https://it.linkedin.com/jobs/view/operatore-co...,2024,LinkedIn,Computer Science / ICT
1,QC Analista Chimico (Lombardia),fortil,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/qc-analista-...,2024,LinkedIn,Engineering
2,Junior Proposal & FP,Gruppo Fervo,"Nova Milanese, Lombardy, Italy",https://it.linkedin.com/jobs/view/junior-propo...,2024,LinkedIn,Social / Communication
3,Impiegato/a Ufficio Accettazione,Blue consulting,"Como, Lombardy, Italy",https://it.linkedin.com/jobs/view/impiegato-a-...,2024,LinkedIn,Other
4,"Environmental, Health, and Safety (EHS) Specia...",Sungrow Europe,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/environmenta...,2024,LinkedIn,Computer Science / ICT
5,Addetto/a Gestione Clienti e Forniture Aziendali,Blue consulting,"Monza, Lombardy, Italy",https://it.linkedin.com/jobs/view/addetto-a-ge...,2024,LinkedIn,Computer Science / ICT
6,Beauty Consultant,Sisley Paris,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/beauty-consu...,2024,LinkedIn,Economics / Business
7,"PRADA Data Entry Intern, Milan",Prada Group,"Milan, Lombardy, Italy",https://it.linkedin.com/jobs/view/prada-data-e...,2024,LinkedIn,Computer Science / ICT
8,Addetto/a Back Office – Azienda Commerciale,Blue consulting,"Bergamo, Lombardy, Italy",https://it.linkedin.com/jobs/view/addetto-a-ba...,2024,LinkedIn,Economics / Business
9,FUNIVIA PONTE DI PIERO-MONTEVIASCO,ATM - Azienda Trasporti Milanesi,"Varese, Lombardy, Italy",https://it.linkedin.com/jobs/view/funivia-pont...,2024,LinkedIn,Other


In [111]:
# Top companies by number of job postings
pd.read_sql_query("""
SELECT company, COUNT(*) AS postings
FROM linkedin_jobs_2024
GROUP BY company
ORDER BY postings DESC
LIMIT 10;
""", conn)



,company,postings
0,Jobbit,152
1,Blue consulting,116
2,Gruppo San Donato,97
3,Mcc service,68
4,KPMG Italy,65
5,Palazzo Parigi Hotel & Grand Spa,63
6,Seingim,59
7,Letuelezioni,54
8,Giucas,51
9,Iper La grande i,48


In [113]:
# Most Common Job Titles (Top 10)
import pandas as pd

query = """
SELECT
    job_title,
    COUNT(*) AS occurrences
FROM linkedin_jobs_2024
WHERE job_title IS NOT NULL
GROUP BY job_title
ORDER BY occurrences DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)


,job_title,occurrences
0,Business Analyst,44
1,Financial Controller,18
2,HR Manager,17
3,Data Scientist,16
4,Business Controller,16
5,Associate Consultant,16
6,Project Manager,15
7,Scrum Master,14
8,Junior Software Engineer,14
9,Cloud Engineer,14


In [114]:
# Most Common Job Titles (with percentage share)
query = """
SELECT
    job_title,
    COUNT(*) AS occurrences,
    ROUND(
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM linkedin_jobs_2024),
        2
    ) AS percentage_share
FROM linkedin_jobs_2024
WHERE job_title IS NOT NULL
GROUP BY job_title
ORDER BY occurrences DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)


,job_title,occurrences,percentage_share
0,Business Analyst,44,0.85
1,Financial Controller,18,0.35
2,HR Manager,17,0.33
3,Data Scientist,16,0.31
4,Business Controller,16,0.31
5,Associate Consultant,16,0.31
6,Project Manager,15,0.29
7,Scrum Master,14,0.27
8,Junior Software Engineer,14,0.27
9,Cloud Engineer,14,0.27


In [115]:
# Verify both tables exist in DB

pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name
0,graduates_milan_2024
1,sqlite_sequence
2,linkedin_jobs_2024


In [116]:
pd.read_sql("PRAGMA table_info(graduates_milan_2024);", conn)
pd.read_sql("PRAGMA table_info(linkedin_jobs_2024);", conn)



,cid,name,type,notnull,dflt_value,pk
0,0,job_title,TEXT,0,None,0
1,1,company,TEXT,0,None,0
2,2,location,TEXT,0,None,0
3,3,url,TEXT,0,None,0
4,4,year,INTEGER,0,None,0
5,5,source,TEXT,0,None,0
6,6,degree_field,TEXT,0,None,0


In [117]:
pd.read_sql("PRAGMA table_info(graduates_milan_2024);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,record_id,INTEGER,0,None,0
1,1,graduation_year,INTEGER,0,None,0
2,2,university_code,INTEGER,0,None,0
3,3,university_name,TEXT,0,None,0
4,4,degree_class_code,TEXT,0,None,0
5,5,course_name,TEXT,0,None,0
6,6,municipality,TEXT,0,None,0
7,7,province_istat_code,INTEGER,0,None,0
8,8,gender_code,TEXT,0,None,0
9,9,graduates,INTEGER,0,None,0


In [118]:
pd.read_sql("PRAGMA table_info(linkedin_jobs_2024);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,job_title,TEXT,0,None,0
1,1,company,TEXT,0,None,0
2,2,location,TEXT,0,None,0
3,3,url,TEXT,0,None,0
4,4,year,INTEGER,0,None,0
5,5,source,TEXT,0,None,0
6,6,degree_field,TEXT,0,None,0


In [119]:
cursor.execute("""
ALTER TABLE graduates_milan_2024
ADD COLUMN degree_field TEXT;
""")
conn.commit()


In [120]:
df_grads = pd.read_sql("SELECT * FROM graduates_milan_2024;", conn)


In [121]:
pd.read_sql("PRAGMA table_info(graduates_milan_2024);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,record_id,INTEGER,0,None,0
1,1,graduation_year,INTEGER,0,None,0
2,2,university_code,INTEGER,0,None,0
3,3,university_name,TEXT,0,None,0
4,4,degree_class_code,TEXT,0,None,0
5,5,course_name,TEXT,0,None,0
6,6,municipality,TEXT,0,None,0
7,7,province_istat_code,INTEGER,0,None,0
8,8,gender_code,TEXT,0,None,0
9,9,graduates,INTEGER,0,None,0


In [122]:
pd.read_sql("PRAGMA table_info(graduates_milan_2024);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,record_id,INTEGER,0,None,0
1,1,graduation_year,INTEGER,0,None,0
2,2,university_code,INTEGER,0,None,0
3,3,university_name,TEXT,0,None,0
4,4,degree_class_code,TEXT,0,None,0
5,5,course_name,TEXT,0,None,0
6,6,municipality,TEXT,0,None,0
7,7,province_istat_code,INTEGER,0,None,0
8,8,gender_code,TEXT,0,None,0
9,9,graduates,INTEGER,0,None,0


In [123]:
df_grads["degree_field"] = df_grads["course_name"].apply(classify_degree_from_title)


In [124]:
df_grads[["course_name", "degree_field"]].head(50)


,course_name,degree_field
0,Infermieristica (Abilitante alla Professione S...,Computer Science / ICT
1,Infermieristica (Abilitante alla Professione S...,Computer Science / ICT
2,Infermieristica Pediatrica (Abilitante alla Pr...,Computer Science / ICT
3,Ostetricia (Abilitante alla Professione Sanita...,Computer Science / ICT
4,Educazione Professionale (Abilitante alla Prof...,Computer Science / ICT
5,Fisioterapia (Abilitante alla Professione Sani...,Computer Science / ICT
6,Fisioterapia (Abilitante alla Professione Sani...,Computer Science / ICT
7,Logopedia (Abilitante alla Professione Sanitar...,Computer Science / ICT
8,Ortottica ed Assistenza Oftalmologica (Abilita...,Computer Science / ICT
9,Podologia (Abilitante alla Professione Sanitar...,Computer Science / ICT


In [125]:
# Write enriched graduates table back to DB
df_grads.to_sql(
    "graduates_milan_2024",
    conn,
    if_exists="replace",
    index=False
)


694

In [126]:
# Load graduates table
grads_df = pd.read_sql("SELECT * FROM graduates_milan_2024", conn)

# Recompute degree_field from course_name
grads_df["degree_field"] = grads_df["course_name"].apply(classify_degree_from_title)

# Save back to DB
grads_df.to_sql(
    "graduates_milan_2024",
    conn,
    if_exists="replace",
    index=False
)


694

In [127]:
pd.read_sql("PRAGMA table_info(linkedin_jobs_2024);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,job_title,TEXT,0,None,0
1,1,company,TEXT,0,None,0
2,2,location,TEXT,0,None,0
3,3,url,TEXT,0,None,0
4,4,year,INTEGER,0,None,0
5,5,source,TEXT,0,None,0
6,6,degree_field,TEXT,0,None,0


In [128]:
# Normalize degree_field in BOTH tables (important)
cursor.execute("""
UPDATE graduates_milan_2024
SET degree_field = LOWER(TRIM(degree_field));
""")

cursor.execute("""
UPDATE linkedin_jobs_2024
SET degree_field = LOWER(TRIM(degree_field));
""")

conn.commit()


In [129]:
#INTEGRATE (JOIN) the two tables
#Row-level integration
integrated_df = pd.read_sql("""
SELECT
    g.graduation_year,
    g.university_name,
    g.course_name,
    g.degree_field,
    g.graduates,
    j.job_title,
    j.company,
    j.location,
    j.year AS job_year
FROM graduates_milan_2024 g
JOIN linkedin_jobs_2024 j
    ON g.degree_field = j.degree_field
WHERE g.graduation_year = 2024
  AND j.degree_field IS NOT NULL;
""", conn)

integrated_df.head()



,graduation_year,university_name,course_name,degree_field,graduates,job_title,company,location,job_year
0,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,3D Configuration & Digital Catalog Intern,Adecco,"Monza e Brianza, Lombardy, Italy",2024
1,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,3D Configuration & Digital Catalog Intern,Adecco,"Monza e Brianza, Lombardy, Italy",2024
2,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,3D Configuration & Digital Catalog Intern,Adecco,"Monza e Brianza, Lombardy, Italy",2024
3,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,3D Configuration & Digital Catalog Intern,Adecco,"Monza e Brianza, Lombardy, Italy",2024
4,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,ADDETTI/E VENDITA - Personale Punto Vendita di...,Eurospin,"Bergamo, Lombardy, Italy",2024


In [130]:
# Supply vs Demand (aggregated integration)
# This is the core analytical result.
supply_demand = pd.read_sql("""
WITH grad AS (
    SELECT degree_field, SUM(graduates) AS total_graduates
    FROM graduates_milan_2024
    WHERE graduation_year = 2024
    GROUP BY degree_field
),
jobs AS (
    SELECT degree_field, COUNT(*) AS total_job_postings
    FROM linkedin_jobs_2024
    GROUP BY degree_field
)
SELECT
    grad.degree_field,
    grad.total_graduates,
    COALESCE(jobs.total_job_postings, 0) AS total_job_postings
FROM grad
LEFT JOIN jobs
    ON grad.degree_field = jobs.degree_field
ORDER BY total_job_postings DESC;
""", conn)


In [131]:
supply_demand_market_check = pd.read_sql("""
WITH grad AS (
    SELECT degree_field, SUM(graduates) AS total_graduates
    FROM graduates_milan_2024
    WHERE graduation_year = 2024
    GROUP BY degree_field
),
jobs AS (
    SELECT degree_field, COUNT(*) AS total_job_postings
    FROM linkedin_jobs_2024
    GROUP BY degree_field
)
SELECT
    grad.degree_field,
    grad.total_graduates,
    COALESCE(jobs.total_job_postings, 0) AS total_job_postings,

    -- Supply / Demand ratio
    ROUND(
        CAST(grad.total_graduates AS FLOAT) /
        NULLIF(jobs.total_job_postings, 0),
        2
    ) AS graduates_per_job,

    -- Demand intensity
    ROUND(
        (COALESCE(jobs.total_job_postings, 0) * 1000.0) /
        grad.total_graduates,
        2
    ) AS jobs_per_1000_graduates,

    -- Qualitative imbalance label
    CASE
        WHEN COALESCE(jobs.total_job_postings, 0) = 0 THEN 'No demand'
        WHEN grad.total_graduates / jobs.total_job_postings > 50 THEN 'Oversupply'
        WHEN grad.total_graduates / jobs.total_job_postings BETWEEN 20 AND 50 THEN 'Balanced'
        ELSE 'High demand'
    END AS market_balance
FROM grad
LEFT JOIN jobs
    ON grad.degree_field = jobs.degree_field
ORDER BY jobs_per_1000_graduates DESC;
""", conn)


In [132]:
supply_demand.head()
supply_demand.sum()


,0
degree_field,computer science / ictothereconomics / busines...
total_graduates,46878
total_job_postings,5200


In [133]:
supply_demand

,degree_field,total_graduates,total_job_postings
0,computer science / ict,10330,1548
1,other,5627,1524
2,economics / business,9994,787
3,social / communication,6297,374
4,engineering,8766,350
5,health,1230,226
6,law / legal studies,1894,130
7,design / architecture,1255,120
8,human resources,249,110
9,psychology,1236,31


In [134]:
# Data Cleaning on Integrated Data
# Cleaning operations to perform
# Remove null degree field
# Check and fix inconsistencies
# Validate the temporal matching (2024 only)
# Remove rows without degree_field (cannot be used for integration)
integrated_df_clean = integrated_df.dropna(subset=["degree_field"])

# Remove duplicates
integrated_df_clean = integrated_df_clean.drop_duplicates()

# Reset index
integrated_df_clean = integrated_df_clean.reset_index(drop=True)

integrated_df_clean.head()



,graduation_year,university_name,course_name,degree_field,graduates,job_title,company,location,job_year
0,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,3D Configuration & Digital Catalog Intern,Adecco,"Monza e Brianza, Lombardy, Italy",2024
1,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,ADDETTI/E VENDITA - Personale Punto Vendita di...,Eurospin,"Bergamo, Lombardy, Italy",2024
2,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,ADDETTO / ADDETTA ACCETTAZIONE SANITARIA,Jobbit,"Milan, Lombardy, Italy",2024
3,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,ADDETTO/A ALLA VENDITA,Kuvera S.p.A. (Carpisa),"Bergamo, Lombardy, Italy",2024
4,2024,Milano,Infermieristica (Abilitante alla Professione S...,computer science / ict,158,AI Agent Evaluation Analyst (Freelance),Mindrift,"Milan, Lombardy, Italy",2024


In [135]:
# STORE integrated_df_clean INTO THE DATABASE
integrated_df_clean.to_sql(
    "integrated_data_2024",
    conn,
    if_exists="replace",
    index=False
)


149172

In [136]:
integrated_df_clean.shape


(149172, 9)

In [137]:
# Exploratory Data Analysis (EDA) on the integrated data

In [139]:
# Missing value profile
integrated_df_clean.isnull().mean() * 100


,0
graduation_year,0.0
university_name,0.0
course_name,0.0
degree_field,0.0
graduates,0.0
job_title,0.0
company,0.0
location,0.0
job_year,0.0


In [140]:
# Distribution of graduates
supply_demand[["degree_field", "total_graduates"]].sort_values(
    "total_graduates", ascending=False
)



,degree_field,total_graduates
0,computer science / ict,10330
2,economics / business,9994
4,engineering,8766
3,social / communication,6297
1,other,5627
6,law / legal studies,1894
7,design / architecture,1255
9,psychology,1236
5,health,1230
8,human resources,249


In [141]:
# Distribution of job demand
supply_demand[["degree_field", "total_job_postings"]].sort_values(
    "total_job_postings", ascending=False
)


,degree_field,total_job_postings
0,computer science / ict,1548
1,other,1524
2,economics / business,787
3,social / communication,374
4,engineering,350
5,health,226
6,law / legal studies,130
7,design / architecture,120
8,human resources,110
9,psychology,31


In [143]:
# Degree fields with the most job demand
integrated_df_clean["degree_field"].value_counts()


,count
degree_field,
computer science / ict,74822
other,37520
economics / business,19152
engineering,8160
social / communication,7482
health,950
design / architecture,504
law / legal studies,406
human resources,104


In [111]:
# “The integrated dataset naturally displays large numerical counts
# because every graduate record associated with a degree field is joined with every j
# ob posting requiring the same degree. This produces a many-to-many relational expansion.
# These counts therefore represent the strength of alignment between degree fields and
# job postings and must not be interpreted as real graduate or job posting counts.
# Real quantities are derived from the original tables,
#  while the integrated table captures compatibility relationships.”

In [163]:
# Top job titles for each degree field
pd.set_option('display.max_rows', None)
integrated_df_clean.groupby("degree_field")["job_title"].value_counts().head(10)


degree_field            job_title                                                  
computer science / ict  Cloud Engineer                                                 627
                        Data Analyst                                                   627
                        Data Scientist                                                 627
                        Junior Software Engineer                                       627
                        Junior Data Analyst                                            418
                        3D Configuration & Digital Catalog Intern                      209
                        ADDETTI/E VENDITA - Personale Punto Vendita di BERGAMO (BG)    209
                        ADDETTO / ADDETTA ACCETTAZIONE SANITARIA                       209
                        ADDETTO/A ALLA VENDITA                                         209
                        AI Agent Evaluation Analyst (Freelance)                        209
Name: count, dtype: int64

In [145]:
# Degree–Industry Alignment

integrated_df_clean.groupby("degree_field")["company"].nunique()
# This shows which degrees connect to more companies → stronger demand.

,company
degree_field,
computer science / ict,234
design / architecture,26
economics / business,131
engineering,66
health,30
human resources,22
law / legal studies,28
other,209
psychology,5


In [146]:
# LOCATION-BASED LABOR MARKET ANALYSIS
integrated_df_clean["location"].value_counts()


,count
location,
"Milan, Lombardy, Italy",94616
"Bergamo, Lombardy, Italy",18778
"Monza, Lombardy, Italy",10213
"Varese, Lombardy, Italy",9723
"Como, Lombardy, Italy",5232
"Monza e Brianza, Lombardy, Italy",2793
"Lecco, Lombardy, Italy",2597
"San Donato Milanese, Lombardy, Italy",1920
"Nova Milanese, Lombardy, Italy",1283


In [153]:
# Degree demand by city
pd.set_option('display.max_rows', None)
integrated_df_clean.groupby(["location", "degree_field"]).size()


location                              degree_field          
Bergamo, Lombardy, Italy              computer science / ict     8151
                                      design / architecture        36
                                      economics / business       2688
                                      engineering                1632
                                      health                       95
                                      human resources              20
                                      law / legal studies          14
                                      other                      5712
                                      social / communication      430
Como, Lombardy, Italy                 computer science / ict     1881
                                      design / architecture        18
                                      economics / business        336
                                      engineering                 408
                                      health                       19
                                      law / legal studies          14
                                      other                      2016
                                      psychology                   24
                                      social / communication      516
Lecco, Lombardy, Italy                computer science / ict     1463
                                      economics / business        336
                                      engineering                 102
                                      health                       38
                                      human resources              12
                                      other                       560
                                      social / communication       86
Lodi, Lombardy, Italy                 computer science / ict      418
                                      other                       224
Milan, Lombardy, Italy                computer science / ict    50160
                                      design / architecture       414
                                      economics / business      12656
                                      engineering                4284
                                      health                      532
                                      human resources              56
                                      law / legal studies         336
                                      other                     21056
                                      psychology                   48
                                      social / communication     5074
Monza e Brianza, Lombardy, Italy      computer science / ict     1881
                                      engineering                 102
                                      health                       38
                                      law / legal studies          14
                                      other                       672
                                      social / communication       86
Monza, Lombardy, Italy                computer science / ict     4807
                                      design / architecture        18
                                      economics / business        896
                                      engineering                 918
                                      health                      114
                                      human resources               4
                                      law / legal studies          28
                                      other                      2912
                                      social / communication      516
Nova Milanese, Lombardy, Italy        computer science / ict      209
                                      economics / business        448
                                      engineering                 204
                                      other                       336
                   

In [152]:
# Gender differences in graduate supply per field


df_grads.groupby(["degree_field", "gender"])["graduates"].sum()


degree_field            gender
Computer Science / ICT  F         5523
                        M         4807
Design / Architecture   F          939
                        M          316
Economics / Business    F         4575
                        M         5419
Engineering             F         2806
                        M         5960
Health                  F          791
                        M          439
Human Resources         F          187
                        M           62
Law / Legal Studies     F         1260
                        M          634
Other                   F         3378
                        M         2249
Psychology              F         1076
                        M          160
Social / Communication  F         4857
                        M         1440
Name: graduates, dtype: int64

In [117]:
###queries from the integrated and cleaned database

In [154]:
# Query 1: Which companies hire the most for each degree field
query1 = """
SELECT
    company,
    degree_field,
    COUNT(*) AS postings
FROM integrated_data_2024
GROUP BY company, degree_field
ORDER BY postings DESC;
"""

company_demand_df = pd.read_sql(query1, conn)
company_demand_df


,company,degree_field,postings
0,Jobbit,computer science / ict,2299
1,Jobbit,other,1680
2,Blue consulting,computer science / ict,1463
3,Dipartimento della Funzione Pubblica,computer science / ict,1463
4,Nexi Italy,computer science / ict,1463
5,BNP Paribas CIB,computer science / ict,1254
6,Codess Sociale Soc Coop,computer science / ict,1254
7,KPMG Italy,computer science / ict,1254
8,EY,computer science / ict,1045
9,Blue consulting,other,1008


In [155]:
# Query 2: Supply–Demand alignment
query2 = """
WITH grad AS (
    SELECT degree_field, SUM(graduates) AS graduates_in_field
    FROM graduates_milan_2024
    WHERE graduation_year = 2024
    GROUP BY degree_field
),
jobs AS (
    SELECT degree_field, COUNT(*) AS matched_job_postings
    FROM linkedin_jobs_2024
    GROUP BY degree_field
)
SELECT
    grad.degree_field,
    grad.graduates_in_field,
    COALESCE(jobs.matched_job_postings, 0) AS matched_job_postings
FROM grad
LEFT JOIN jobs
    ON grad.degree_field = jobs.degree_field
ORDER BY matched_job_postings DESC;
"""


supply_demand_df = pd.read_sql(query2, conn)
supply_demand_df


,degree_field,graduates_in_field,matched_job_postings
0,computer science / ict,10330,1548
1,other,5627,1524
2,economics / business,9994,787
3,social / communication,6297,374
4,engineering,8766,350
5,health,1230,226
6,law / legal studies,1894,130
7,design / architecture,1255,120
8,human resources,249,110
9,psychology,1236,31


In [156]:
query2 = """
WITH grad AS (
    SELECT degree_field, SUM(graduates) AS graduates_in_field
    FROM graduates_milan_2024
    WHERE graduation_year = 2024
    GROUP BY degree_field
),
jobs AS (
    SELECT degree_field, COUNT(*) AS matched_job_postings
    FROM linkedin_jobs_2024
    GROUP BY degree_field
),
totals AS (
    SELECT
        (SELECT SUM(graduates_in_field) FROM grad) AS total_graduates,
        (SELECT SUM(matched_job_postings) FROM jobs) AS total_jobs
)
SELECT
    grad.degree_field,
    grad.graduates_in_field,
    jobs.matched_job_postings,

    ROUND(grad.graduates_in_field * 100.0 / totals.total_graduates, 2) AS pct_graduates,
    ROUND(jobs.matched_job_postings * 100.0 / totals.total_jobs, 2) AS pct_job_postings,

    ROUND(
        (jobs.matched_job_postings * 1.0 / grad.graduates_in_field) * 100, 2
    ) AS demand_supply_ratio

FROM grad
LEFT JOIN jobs ON grad.degree_field = jobs.degree_field
CROSS JOIN totals
ORDER BY pct_job_postings DESC;
"""
supply_demand_df = pd.read_sql(query2, conn)
supply_demand_df


,degree_field,graduates_in_field,matched_job_postings,pct_graduates,pct_job_postings,demand_supply_ratio
0,computer science / ict,10330,1548,22.04,29.77,14.99
1,other,5627,1524,12.00,29.31,27.08
2,economics / business,9994,787,21.32,15.13,7.87
3,social / communication,6297,374,13.43,7.19,5.94
4,engineering,8766,350,18.70,6.73,3.99
5,health,1230,226,2.62,4.35,18.37
6,law / legal studies,1894,130,4.04,2.50,6.86
7,design / architecture,1255,120,2.68,2.31,9.56
8,human resources,249,110,0.53,2.12,44.18
9,psychology,1236,31,2.64,0.60,2.51


In [160]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name
0,sqlite_sequence
1,linkedin_jobs_2024
2,graduates_milan_2024
3,integrated_data_2024


In [121]:
# Data Quality Assessment

In [157]:
# Completeness
integrated_df.isnull().mean() * 100
# The integrated dataset shows 0% missing values across all fields.
# This is because:

# graduate data (municipal dataset) is complete and well-structured

# LinkedIn scraper filtered out records with missing job titles or locations

# the join condition (degree_field) was normalized and made complete before integration

,0
graduation_year,0.0
university_name,0.0
course_name,0.0
degree_field,0.0
graduates,0.0
job_title,0.0
company,0.0
location,0.0
job_year,0.0


In [158]:
# Consistency
pd.Series(
    supply_demand["degree_field"].unique()
)
# Degree fields were normalized and aligned across heterogeneous sources to ensure semantic consistency during integration.

,0
0,computer science / ict
1,other
2,economics / business
3,social / communication
4,engineering
5,health
6,law / legal studies
7,design / architecture
8,human resources
9,psychology


In [124]:
# Accuracy
# Graduate counts come from official municipal open data, while job demand is derived from web scraping and represents an observable sample rather than the full market.

In [159]:
# Timeliness
integrated_df["graduation_year"].unique()
integrated_df["job_year"].unique()
# Both datasets were aligned on year 2024, ensuring temporal consistency.

array([2024])

In [126]:
# A relational database was chosen because the datasets are structured and require
#  SQL JOIN operations to integrate graduation data with job market postings.
#   The relational model supports ACID guarantees, schema enforcement,
#    efficient querying, and high-quality data cleaning—making it the ideal
#    choice for analyzing the relationship between
# higher-education outputs and job demand in Milan.